In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
num_classes = 2
learning_rate = 0.001
dropout_rate = 0.5
batch_size = 32
num_epochs = 25

In [4]:
class ResNetIrisTumor(nn.Module):
    def __init__(self, num_classes, dropout_rate):
        super(ResNetIrisTumor, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.resnet.fc.in_features, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)

In [5]:
model = ResNetIrisTumor(num_classes=num_classes, dropout_rate=dropout_rate)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 118MB/s]


In [6]:
class CustomDataset(Dataset):
    def __init__(self, folder_path, label, transform=None):
        self.folder_path = folder_path
        self.label = label
        self.transform = transform
        self.image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.label

In [7]:
tumor_path = "/content/drive/MyDrive/iristumordataset/YES"
no_tumor_path = "/content/drive/MyDrive/iristumordataset/NO"

In [8]:
tumor_dataset = CustomDataset(tumor_path, label=1, transform=transform)
no_tumor_dataset = CustomDataset(no_tumor_path, label=0, transform=transform)
combined_dataset = ConcatDataset([tumor_dataset, no_tumor_dataset])
train_size = int(0.8 * len(combined_dataset))
val_size = len(combined_dataset) - train_size
train_data, val_data = random_split(combined_dataset, [train_size, val_size])

In [11]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
num_epochs=10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to('cpu'), labels.to('cpu')
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = 100. * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to('cpu'), labels.to('cpu')

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_epoch_loss = val_loss / len(val_loader.dataset)
    val_epoch_acc = 100. * val_correct / val_total
    print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.2f}%')



Epoch 1/10, Loss: 0.0129, Accuracy: 99.15%
Validation Loss: 0.1277, Validation Accuracy: 90.00%
Epoch 2/10, Loss: 0.0023, Accuracy: 100.00%
Validation Loss: 0.4454, Validation Accuracy: 93.33%
Epoch 3/10, Loss: 0.0073, Accuracy: 100.00%
Validation Loss: 0.4051, Validation Accuracy: 93.33%
Epoch 4/10, Loss: 0.0030, Accuracy: 100.00%
Validation Loss: 0.2752, Validation Accuracy: 93.33%
Epoch 5/10, Loss: 0.0030, Accuracy: 100.00%
Validation Loss: 0.1722, Validation Accuracy: 96.67%
Epoch 6/10, Loss: 0.0024, Accuracy: 100.00%
Validation Loss: 0.1605, Validation Accuracy: 96.67%
Epoch 7/10, Loss: 0.0004, Accuracy: 100.00%
Validation Loss: 0.1821, Validation Accuracy: 93.33%
Epoch 8/10, Loss: 0.0002, Accuracy: 100.00%
Validation Loss: 0.2149, Validation Accuracy: 90.00%
Epoch 9/10, Loss: 0.0059, Accuracy: 100.00%
Validation Loss: 0.2521, Validation Accuracy: 93.33%
Epoch 10/10, Loss: 0.0002, Accuracy: 100.00%
Validation Loss: 0.6532, Validation Accuracy: 86.67%


In [12]:
torch.save(model.state_dict(), '/content/drive/MyDrive/iristumordataset/Resnet.pth')


In [13]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 57.1 MB/s eta 0:00:00


In [14]:
import torch

dummy_input = torch.randn(1, 3, 224, 224)  # Adjust dimensions as needed
torch.onnx.export(model, dummy_input, '/content/drive/MyDrive/iristumordataset/Resnet.onnx')


In [15]:
scripted_model = torch.jit.script(model)  # Or torch.jit.trace for tracing
scripted_model.save('/content/drive/MyDrive/iristumordataset/Resnet.pt')


In [16]:
import torch
from torch.quantization import quantize_dynamic

# Apply dynamic quantization
quantized_model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
torch.save(quantized_model.state_dict(), '/content/drive/MyDrive/iristumordataset/Resnet_quantized.pth')


In [17]:
import torch.nn.utils.prune as prune

# Apply pruning to layers
# Changed model.fc to model.resnet.fc to access the fully connected layer
prune.random_unstructured(model.resnet.fc[1], name="weight", amount=0.5)  # Example: prune 50% weights of the linear layer within the sequential
torch.save(model.state_dict(), '/content/drive/MyDrive/iristumordataset/Resnet_pruned.pth')

In [18]:
torch.save(model.state_dict(), '/content/drive/MyDrive/iristumordataset/Resnet_weights_only.pth')


In [21]:
import torch
import torch.nn as nn
from torchvision import models

# Define and load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Adjust output layer for your dataset
model = model.to(device)
model.eval()  # Ensure the model is in evaluation mode for exporting

# Dummy input for exporting (match your model's input size)
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Export to ONNX
onnx_path = '/content/drive/MyDrive/iristumordataset/Resnet_optimized.onnx'
torch.onnx.export(model, dummy_input, onnx_path, opset_version=11, input_names=['input'], output_names=['output'])
print(f"ONNX model saved to {onnx_path}")

# Export to TorchScript
scripted_model = torch.jit.script(model)
scripted_path = '/content/drive/MyDrive/iristumordataset/Resnet_optimized.pt'
scripted_model.save(scripted_path)
print(f"TorchScript model saved to {scripted_path}")


ONNX model saved to /content/drive/MyDrive/iristumordataset/Resnet_optimized.onnx
TorchScript model saved to /content/drive/MyDrive/iristumordataset/Resnet_optimized.pt


In [22]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [23]:
import streamlit as st
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import onnxruntime as ort

# Load PyTorch model
@st.cache_resource
def load_torch_model(model_path):
    model = torch.jit.load(model_path)
    model.eval()
    return model

# Load ONNX model
@st.cache_resource
def load_onnx_model(model_path):
    session = ort.InferenceSession(model_path)
    return session

# Preprocessing function
def preprocess_image(image, target_size=(224, 224)):
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

# Prediction function for PyTorch
def predict_torch(image_tensor, model):
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
    return predicted.item()

# Prediction function for ONNX
def predict_onnx(image_tensor, session):
    input_name = session.get_inputs()[0].name
    output = session.run(None, {input_name: image_tensor.numpy()})
    predicted = np.argmax(output[0], axis=1)
    return predicted[0]

# Streamlit UI
st.markdown(
    """
    <style>
    .main {
        background-color: pink;
        padding: 5rem;
    }
    .title {
        text-align: center;
        color: black;
        font-size: 36px;
        font-weight: bold;
    }
    </style>
    """, unsafe_allow_html=True
)

st.title("Iris Tumor Classification")

# Customizing title style
st.markdown('<h1 class="title">Iris Tumor Classification</h1>', unsafe_allow_html=True)

st.write("Upload an image of the iris to classify as tumor or no tumor.")

# Sidebar for model selection
model_type = st.sidebar.selectbox("Choose a model", ["PyTorch", "ONNX"])

# Model paths
torch_model_path = "/content/drive/MyDrive/iristumordataset/Resnet.pt"
onnx_model_path = "/content/drive/MyDrive/iristumordataset/Resnet.onnx"

# Load the selected model
if model_type == "PyTorch":
    model = load_torch_model(torch_model_path)
else:
    session = load_onnx_model(onnx_model_path)

# File uploader
uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
if uploaded_file:
    # Display uploaded image
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Preprocess the image
    st.write("Processing the image...")
    image_tensor = preprocess_image(image)

    # Make predictions
    if model_type == "PyTorch":
        prediction = predict_torch(image_tensor, model)
    else:
        # Convert Torch tensor to NumPy array for ONNX
        prediction = predict_onnx(image_tensor, session)

    # Display the prediction
    if prediction == 1:
        st.success("Prediction: Tumor Detected")
    else:
        st.success("Prediction: No Tumor Detected")


app.py has been saved!


In [ ]:
!streamlit run app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.101.74:8501

